<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/tpu/fine_tune_tpu_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Xception TPUs (Colab)&nbsp;<img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50">

### Enabling and testing the TPU

In [1]:
import re
import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt

tf.__version__

'2.3.0'

In [2]:
#@title Enable TPU
use_tpu = True #@param ["False", "True"] {type:"raw"}

if use_tpu:
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tpu_spec = tpu.cluster_spec().as_dict()['worker']
    print('Running on TPU:', tpu_spec)
  except ValueError:
    print('ERROR: Not connected to a TPU.')

  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU: ['10.27.206.210:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.27.206.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.27.206.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)



## Input data

Our input data is stored on Google Cloud Storage. To more fully use the parallelism TPUs offer us, and to avoid bottlenecking on data transfer, we've stored our input data in TFRecord files, 230 images per file.

Below, we make heavy use of `tf.data.experimental.AUTOTUNE` to optimize different parts of input loading.

All of these techniques are a bit overkill for our (small) dataset, but demonstrate best practices for using TPUs.

In [3]:
AUTO = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = [331, 331]
GCS_PATTERN = 'gs://flowers-public/tfrecords-jpeg-331x331/*.tfrec'
VALIDATION_SPLIT = 0.19
BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync
SHUFFLE = 2048
print('Batch Size:', BATCH_SIZE)

Batch Size: 128


In [4]:
filenames = tf.io.gfile.glob(GCS_PATTERN)
split = len(filenames) - int(len(filenames) * VALIDATION_SPLIT)
TRAIN_FNS = filenames[:split]
VALIDATION_FNS = filenames[split:]
print('Train len:', len(TRAIN_FNS), ' ', 'Validation len:', len(VALIDATION_FNS))

NameError: ignored

In [ ]:
tf.io.FixedLenFeature([], tf.string)

In [ ]:
def parse_tfrecord(example):
  features = {
    'image': tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
    'class': tf.io.FixedLenFeature([], tf.int64),  # shape [] means scalar
    'one_hot_class': tf.io.VarLenFeature(tf.float32),
  }
  example =tf.io.parse_single_example(example, features)
  decoded =tf.image.decode_jpeg(example['image'], channels=3)
  normalized = tf.cast(decoded, tf.float32) / 255.0
  image_tensor = tf.reshape(normalized, [*IMAGE_SIZE, 3])
  one_hot_class = tf.reshape(tf.sparse.to_dense(example['one_hot_class']), [5])
  return image_tensor, one_hot_class

In [ ]:
def load_dataset(filenames):
  # Read from TFRecords. For optimal performance, 
  # we interleave reads from multiple files.
  records = tf.data.TFRecordDataset(
      filenames, 
      num_parallel_reads=AUTO
  )
  return records.map(parse_tfrecord, num_parallel_calls=AUTO)

In [ ]:
# create some additional training images by randomly flipping and
# increasing/decreasing the saturation of images in the training set
def data_augment(image, one_hot_class):
  modified = tf.image.random_flip_left_right(image)
  modified = tf.image.random_saturation(modified, 0, 2)
  return modified, one_hot_class

In [ ]:
def optimize_batch(tfds):
  return tfds.batch(BATCH_SIZE).prefetch(AUTO)

In [ ]:
def get_training_dataset():
  dataset = load_dataset(TRAIN_FNS)
  augmented = dataset.map(data_augment, num_parallel_calls=AUTO)
  return optimize_batch(augmented.repeat().shuffle(SHUFFLE))

In [ ]:
training_dataset = get_training_dataset()
validation_dataset = optimize_batch(load_dataset(validation_fns))